In [1]:
from augment import *
import glob
import os
import glob
import scipy
import random
import bisect
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import signal
from scipy.signal import find_peaks
import matplotlib.pyplot as plt
from preprocess import *                         #ماژول استخراج پنجره ها
from data import *                               #ماژول محلی ورود داده ها
from augment import *                            #ماژول های داده افزایی
from GAN import *                  
from lstm_cnn import *                           #ماژول های داده افزایی با lstm_cnn
import augment
import importlib
importlib.reload(augment)
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import BatchNormalization
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import make_classification
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report,recall_score,precision_score
from tensorflow.keras import callbacks
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Convolution2D,Conv1D,Dropout,MaxPooling1D,MaxPooling2D,Flatten,Dense
from tensorflow.keras import backend as K
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model
cwd = os.getcwd() #

In [2]:
def dstnc(dt2,dt1): #فاصله و واریانس بردار فاصله ی هر کدام از ردیف های دیتاست(آگمنت)1 از (کل)2
    dist,vrnc,i,h,w=[],[],0,np.shape(dt1)[0],np.shape(dt1)[1]
    end=min(len(dt2[0]),len(dt1[0]))
    for i in range (h):
        ds1 = dt2-dt1[i,:end] #.reshape(-1,1) #distance of every training instance
        dist_array=(ds1*10)**3  #معیار فاصله با حساسیت به توان 3 (حفظ علامت و تاثیر بیشتر تفاوت زیاد در نقاط)
        vr=np.var(ds1)
        ds1=min(np.sum(np.absolute(dist_array),axis=1))
        #print('dist2 = ', dist2)
        vrnc.append(vr)
        dist.append(ds1)
    variance=np.reshape(vrnc,(-1,1))
    distnc=np.reshape(dist,(-1,1))
    return(distnc,variance)

cls_num=12
ecg=np.array([])
for i in range (1,cls_num+1):
    vars()['ecg'+str(i)],vars()['ecg_tst'+str(i)]=EOGVerticalSignal(i)         #فراخوانی داده های اصلی
    ecg=np.append(ecg,vars()['ecg'+str(i)])
wdth=int(len(ecg1[0]))
ecg=np.reshape(ecg,(int(len(ecg)/wdth),wdth))  #base data windows
print('\nfinal_train_shap=',np.shape(ecg))
os.chdir(cwd)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\util\_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


first_train_shape= (362, 1251)
first_test_shape= (362, 1251)
classes_quantity= 12
tr_lbls=	 {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12}
Count_labels= [30 30 30 30 30 31 31 30 30 30 30 30]
max(train_feature_Altitude)= 2540.7
min(train_feature_Altitude)= -923.79
first_train_sample=
 [   1.       -5.7032   -5.8955 ... -113.16   -112.85   -112.94  ]

final_train_shap= (362, 481)


In [3]:
base_wn=np.array([])
windws=np.array([])

for cls in range (1,cls_num+1):                   #ساخت پنجره های داده های آموزش اصلی
    vars()['wndws'+str(cls)]=np.array(vars()['ecg'+str(cls)])
    base_wn=np.append(base_wn,vars()['wndws'+str(cls)])
base_wn=np.reshape(base_wn,(int(len(base_wn)/wdth),wdth))  #base data windows
print("shape wndws1=", np.shape(wndws1))
print("shape base_wn=", np.shape(base_wn))

furir_windws0=np.fft.fft(np.array(base_wn[:,:-1]))
print("shape furir_windws0=", np.shape(furir_windws0))
furir_windws=np.concatenate((np.array(furir_windws0),np.transpose([base_wn[:,-1]])),axis=1)   
print("shape furir_windws=", np.shape(furir_windws))

smpl_rte=len(base_wn[0])-1                 #len(ecg1[0])  # در ماژول ها نیز همین مقدار ثبت شده
print("smpl_rte=",smpl_rte)
mx_wndws=np.max(np.bincount(np.int16(base_wn[:,-1])))
print("mx_wndws=",mx_wndws)

shape wndws1= (30, 481)
shape base_wn= (362, 481)
shape furir_windws0= (362, 480)
shape furir_windws= (362, 481)
smpl_rte= 480
mx_wndws= 31


In [4]:
path=('{}\AugEvlV'.format(cwd)) #create folder 
if not os.path.isdir(path):
    os.mkdir(path)

def addqlty(base_wn_dt,data,mthd,i):               
    cls=int(i)
    c,alpha,PL=0,1,1
    #from glob import glob
    cwd = os.getcwd() #
    adrs=cwd+'/Aug/aug'+str(i)+str(mthd)+'.csv'
    #alldata = np.array([])                    # List for storing all the data
    train_labels=np.array(base_wn_dt[:,-1])
    tr=np.array(base_wn_dt[:,:-1])
    indx_pstv=np.where(base_wn_dt[:,-1] == i)
    P=np.array(base_wn_dt[indx_pstv])
    indx_ngtv=np.where(base_wn_dt[:,-1] != i)
    N=np.array(base_wn_dt[indx_ngtv])

    #data = np.array(data)
    aug=np.array(data)
    pnts=int(np.shape(data)[1])
    
    #aug=data[data[:,-2].argsort()]        # Positive data Sorted Base Distance
    trl=np.array(train_labels)
    for j in range (len(trl)):
        if train_labels[j]==i :           # !!!!! == (Paper GDO)
            trl[j]=1
        else :
            trl[j]=0
    trlt=np.array([trl]).T
    #mn_ds_pstv=np.reshape(aug[:,-2],(-1,1)) #min dis to positive class (submited in -2 column)
    print('shape and aug[0]=',np.shape(aug[0]),aug[0])
    print('shape P and P[0]=',np.shape(P),P[0])    
    qlty1,qlty2=mn_ds_pstv,vr_P=dstnc(P[:,1:-1],aug[:,1:-1])  #min distance to negative class
    new1=np.append(aug,qlty1,axis=1)
    new2=np.append(new1,qlty2,axis=1)
    mn_ds_ngtv,vr_N=dstnc(N[:,1:-1],aug[:,1:-1])              #min distance to negative class
    qlty3=mn_ds_pstv/mn_ds_ngtv                               #Smaller is Better
    qlty4=np.reshape(valuation(aug[:,:-1],tr,trlt),(-1,1))   #line 272
    new3=np.append(new2,qlty3,axis=1)
    new4=np.append(new3,qlty4,axis=1)
    for j in range (len(trl)):
        if train_labels[j]!=i :                          # !!!!! == (!= Inverse Paper GDO)
            trl[j]=1
        else :
            trl[j]=0
    trlt=np.array([trl]).T
    qlty5=np.reshape(valuation(aug[:,:-1],tr,trlt),(-1,1))   #line 272
    new5=np.append(new4,qlty5,axis=1)
    print('class=',i,'shape(new5)=',np.shape(new5),end='\t')
    print('mx=',np.max(new5[:,1:-7]),end='\t')
    print('mn=',np.min(new5[:,1:-7]))
    pd.DataFrame(new5[:,:]).to_csv('{}/AugEvlV/AugEvl{}{}.csv'.format(cwd,cls,mthd))
    print()

In [5]:
for i in range (1,cls_num+1):                           #تعریف آرایه ی پنجره های کلاس ها
    vars()['ecg_aug'+str(i)+'00']=np.empty((0,smpl_rte+1), float) #Scale    
    vars()['ecg_aug'+str(i)+'10']=np.empty((0,smpl_rte+1), float) #Scale
    vars()['ecg_aug'+str(i)+'20']=np.empty((0,smpl_rte+1), float) #Mag Warp
    vars()['ecg_aug'+str(i)+'30']=np.empty((0,smpl_rte+1), float) #TimWarp
    vars()['ecg_aug'+str(i)+'40']=np.empty((0,smpl_rte+1), float) #GDO
    vars()['ecg_aug'+str(i)+'50']=np.empty((0,smpl_rte+1), float) #frequence
    vars()['ecg_aug'+str(i)+'60']=np.empty((0,smpl_rte+1), float) #GAN
    vars()['ecg_aug'+str(i)+'61']=np.empty((0,smpl_rte+1), float)
    vars()['ecg_aug'+str(i)+'62']=np.empty((0,smpl_rte+1), float)
    vars()['ecg_aug'+str(i)+'70']=np.empty((0,smpl_rte+1), float) #LSTM
    vars()['ecg_aug'+str(i)+'71']=np.empty((0,smpl_rte+1), float)
    vars()['ecg_aug'+str(i)+'72']=np.empty((0,smpl_rte+1), float)

In [6]:
for i in range (1,cls_num+1):                           #تعریف آرایه ی پنجره های کلاس ها
    vlum_sig=10*mx_wndws-len(vars()['wndws'+str(i)])
    
    vars()['ecg_aug'+str(i)+'10']=np.append(vars()['ecg_aug'+str(i)+'10'],sclRaw(ecg,i,int(vlum_sig),mag=.2),axis=0)#0.1
    addqlty(base_wn,vars()['ecg_aug'+str(i)+'10'],10,i)
    print(str(i)+'10 =',len(vars()['ecg_aug'+str(i)+'10']))
    
    vars()['ecg_aug'+str(i)+'20']=np.append(vars()['ecg_aug'+str(i)+'20'],MgWrpRaw(ecg,i,int(vlum_sig),mag=.2),axis=0)
    addqlty(base_wn,vars()['ecg_aug'+str(i)+'20'],20,i)
    print(str(i)+'20 =',len(vars()['ecg_aug'+str(i)+'20']))
    
    vars()['ecg_aug'+str(i)+'30']=np.append(vars()['ecg_aug'+str(i)+'30'],TimWrpRaw(ecg,i,int(vlum_sig),mag=.2),axis=0)#.1
    addqlty(base_wn,vars()['ecg_aug'+str(i)+'30'],30,i)
    print(str(i)+'30 =',len(vars()['ecg_aug'+str(i)+'30']))
    
    vars()['ecg_aug'+str(i)+'40']=np.append(vars()['ecg_aug'+str(i)+'40'],GDORaw(ecg,i,int(vlum_sig),mag=.2),axis=0)
    addqlty(base_wn,vars()['ecg_aug'+str(i)+'40'],40,i)
    print(str(i)+'40 =',len(vars()['ecg_aug'+str(i)+'40']))
    
    vars()['ecg_aug'+str(i)+'50']=np.append(vars()['ecg_aug'+str(i)+'50'],frqncRaw(furir_windws,i,int(vlum_sig),mag=.2),axis=0)
    addqlty(base_wn,vars()['ecg_aug'+str(i)+'50'],50,i)
    print(str(i)+'50 =',len(vars()['ecg_aug'+str(i)+'50']))

shape and aug[0]= (481,) [-1.34500000e+02  1.81718750e+01 -2.83437500e+01  2.88476562e+00
 -1.50781250e+01 -1.62304688e+00 -1.13515625e+01 -8.12011719e-01
 -1.11562500e+01 -1.57714844e+00 -8.33593750e+00 -3.25585938e+00
 -7.71093750e+00 -8.80859375e-01 -6.32421875e+00 -7.58666992e-02
 -1.69238281e+00  2.55664062e+00 -1.28417969e+00  1.78710938e+00
 -1.95800781e+00  2.27813721e-02 -4.01953125e+00 -2.00195312e+00
 -3.22851562e+00  2.19482422e-01 -2.38085938e+00 -1.28906250e-01
 -1.38281250e+00  3.40429688e+00  2.28125000e+00  4.38671875e+00
  2.85546875e+00  4.12109375e+00  2.33007812e+00  4.37500000e+00
  1.92675781e+00  4.83984375e+00  6.08203125e+00  8.87500000e+00
  7.98437500e+00  1.00312500e+01  8.35156250e+00  1.02656250e+01
  9.88281250e+00  1.04375000e+01  9.00781250e+00  9.92187500e+00
  8.64062500e+00  9.96875000e+00  8.32031250e+00  1.00312500e+01
  8.42187500e+00  8.74218750e+00  7.84375000e+00  7.69140625e+00
  6.96875000e+00  7.42578125e+00  5.68359375e+00  5.77734375e+00


class= 1 shape(new5)= (283, 486)	mx= 1928.0	mn= -1300.0

110 = 283


C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3419: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\_methods.py:188: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


shape and aug[0]= (481,) [-1.98710768e+01  6.93931455e-01 -6.29103504e+00 -2.14714590e+00
 -6.85714569e+00 -4.25960567e+00 -5.90813816e+00 -4.16857998e+00
 -6.14170035e+00 -4.33821605e+00 -5.75778543e+00 -5.05677991e+00
 -4.19535546e+00 -1.54246046e+00 -2.93736570e+00 -2.13466263e+00
 -1.96895887e+00  8.12547314e-01  1.82616953e+00  3.65310492e+00
  4.23367762e+00  5.31209646e+00  5.44413971e+00  6.36154496e+00
  6.70813893e+00  6.60182881e+00  6.60941866e+00  6.69413867e+00
  5.42147339e+00  5.17016622e+00  3.47544597e+00  1.29250411e+00
  1.70631448e+00  1.25401514e+00  1.60186708e+00  2.92467474e+00
  2.05356776e+00  2.87395864e+00  4.58964625e+00  5.76082277e+00
  4.25783453e+00  3.70026674e+00  2.00643552e+00  3.32856418e+00
  1.68425122e+00  3.47482220e+00  2.89507203e+00  2.57900774e+00
  2.12695841e+00  2.59307233e+00  8.58829423e-01 -1.36863609e+00
 -1.22931824e+00 -8.05323844e-01 -4.01916249e-02 -4.84675364e-01
 -1.83143871e-01  2.15391331e-02  6.04191622e-01  1.96290322e-01


class= 1 shape(new5)= (283, 486)	mx= 1565.0595995915005	mn= -1156.4130078501912

120 = 283
shape and aug[0]= (481,) [ 2.45369761e+01  1.53013234e+01  7.21389276e+00  9.13200382e+00
  1.03316834e+01  9.01542720e+00  8.65269377e+00  1.00042857e+01
  1.02184752e+01  9.33409749e+00  9.74910566e+00  1.07874279e+01
  9.95155631e+00  8.85324385e+00  8.51441293e+00  8.06807381e+00
  7.33266013e+00  7.47771203e+00  8.50186686e+00  8.33260219e+00
  7.75586814e+00  7.44088239e+00  6.78663806e+00  5.25635858e+00
  4.63912919e+00  4.71458509e+00  4.98976316e+00  5.21785860e+00
  5.12447611e+00  5.04508173e+00  4.97835092e+00  4.54494878e+00
  4.21449547e+00  4.39569753e+00  4.76905109e+00  5.26507542e+00
  5.13033665e+00  4.57378655e+00  3.39458229e+00  1.91224689e+00
  5.76267824e-01 -1.30776353e-01 -1.16621414e+00 -2.27909029e+00
 -3.03482249e+00 -4.23435536e+00 -5.66485388e+00 -6.81740056e+00
 -7.26642448e+00 -7.38120516e+00 -8.40773989e+00 -9.20223888e+00
 -9.76343011e+00 -9.84085459e+00 -1.010

...

class= 12 shape(new5)= (283, 486)	mx= 980.5	mn= -250.25

1210 = 283
shape and aug[0]= (481,) [-9.37839353e+00 -1.59143900e+00 -3.65221926e+00 -1.45225354e+00
 -1.93958949e+00 -1.18113882e+00 -2.84081712e+00 -2.50295878e+00
 -3.75966667e+00 -3.50587949e+00 -3.90981295e+00 -3.29170627e+00
 -3.52569055e+00 -3.18245292e+00 -3.56803419e+00 -3.17667729e+00
 -3.29542836e+00 -3.04193010e+00 -2.09640174e+00 -1.97925071e+00
 -1.60416784e-01  9.28788387e-01  9.77480193e-01  1.31209499e+00
  1.38859842e+00  1.98411136e+00  2.55496930e+00  2.49015026e+00
  2.29361918e+00  2.52153191e+00  3.16948185e+00  3.77157673e+00
  3.67246410e+00  2.93271000e+00  2.84012962e+00  3.01593812e+00
  3.13041540e+00  4.01553510e+00  4.05596493e+00  4.77122088e+00
  5.18788832e+00  3.37429442e+00  4.09014797e+00  3.77276298e+00
  8.26794858e-01 -8.76616122e-02 -2.61521994e+00 -1.30270128e+00
  9.52397645e-02 -5.92910513e-02 -9.11677290e-01 -2.44796504e-01
  1.09340700e+00  1.52522558e+00  1.92504075e+00  2.09418725e+

class= 12 shape(new5)= (283, 486)	mx= 802.6639047621904	mn= -239.39969289509582

1220 = 283
shape and aug[0]= (481,) [-1.73323463e+00 -1.27185454e+00 -1.42623213e+00  4.12570279e-01
  8.82653790e-02 -1.28445015e+00 -1.00017296e+00 -2.07620722e+00
 -2.64406274e+00  4.00220217e-01  2.01899105e+00  3.01249292e+00
  3.59382077e+00  3.75481959e+00  4.86020969e+00  6.04203708e+00
  6.95466778e+00  5.93124902e+00  7.32820826e+00  9.31460485e+00
  8.10630521e+00  8.30253412e+00  9.12490311e+00  7.20358590e+00
  6.73538232e+00  7.98492977e+00  5.64845696e+00  3.04893451e+00
  3.68351212e+00  3.35158269e+00  1.75419042e+00  1.38594414e+00
  1.56938736e+00  2.50465472e+00  3.28804017e+00  2.20431650e+00
  2.06846466e+00  4.54206873e+00  6.01661125e+00  5.96307113e+00
  6.36875950e+00  7.24568028e+00  7.04890191e+00  6.01421911e+00
  5.31977266e+00  4.85189131e+00  4.39804859e+00  3.95318458e+00
  3.65720592e+00  3.46419286e+00  3.66992352e+00  4.20910320e+00
  3.85387952e+00  2.49312110e+00  1.68

class= 12 shape(new5)= (283, 486)	mx= 666.4330455917263	mn= -112.2487018302175

1230 = 283
shape and aug[0]= (481,) [-2.91790418e+00 -3.56466608e+00 -4.14049112e+00 -4.64225280e+00
 -3.39277986e+00 -3.31619271e+00 -1.83080764e+00 -1.22119347e+00
  1.56396263e+00  4.73671707e+00  4.63953751e+00  5.23433754e+00
  4.51769666e+00  5.32479202e+00  4.86589473e+00  5.03223270e+00
  3.52892206e+00  3.25117343e+00  2.02271788e+00  3.09470825e+00
  4.50171743e+00  4.59042983e+00  4.87764773e+00  4.27753466e+00
  3.79299592e+00  2.44620499e+00  2.62385125e+00  1.52579278e+00
  8.93565703e-01  3.82126109e-01  1.17571074e-01  9.57889261e-02
 -1.26544568e-01 -1.99567622e+00 -5.26958480e+00 -5.96989184e+00
 -6.01452565e+00 -6.44060537e+00 -6.98936744e+00 -7.12024138e+00
 -7.15448780e+00 -6.52572636e+00 -5.77861659e+00 -4.39330932e+00
 -4.16341478e+00 -9.63404364e-01  8.52648303e-01  1.73059706e+00
  2.83005114e+00  6.69119072e-01  2.96266613e-01  2.72855044e-01
  5.98597689e-01  7.24119797e-01  1.601

class= 12 shape(new5)= (283, 486)	mx= 843.2466979892446	mn= -231.85091698369638

1240 = 283
shape and aug[0]= (481,) [ 1.61093750e+01 -6.12890625e+00  9.21386719e-01 -2.81445312e+00
 -1.62597656e-01 -3.40820312e+00 -3.30810547e-01 -6.48437500e-01
  9.75585938e-01 -1.60278320e-01 -4.72900391e-01 -1.36816406e+00
 -2.74023438e+00 -1.10644531e+00 -2.68554688e+00 -2.54882812e+00
 -2.31445312e+00  4.32434082e-02  6.58691406e-01  4.42382812e-01
  7.13378906e-01 -1.53710938e+00 -1.31103516e-01 -1.03515625e+00
 -1.84472656e+00 -2.38671875e+00 -3.83007812e+00 -4.60937500e+00
 -2.25781250e+00 -2.08398438e+00 -3.06835938e+00 -3.73046875e+00
 -3.07031250e+00 -2.43750000e+00  7.11425781e-01 -2.21093750e+00
 -2.35742188e+00 -4.17578125e+00 -4.69140625e+00 -3.66015625e+00
 -3.41601562e+00 -2.29882812e+00 -3.36914062e+00 -4.03906250e+00
 -2.36914062e+00 -1.58203125e+00 -2.56054688e+00 -2.46289062e+00
 -2.21484375e+00 -1.81250000e+00  2.39062500e+00  4.03515625e+00
  4.36328125e+00  4.37109375e+00  4.50

class= 12 shape(new5)= (283, 486)	mx= 680.0	mn= -208.75

1250 = 283
